## Cat Facts
This notebook shows how to create, run, and use output from the Cat Facts assistant! This assistant uses a free demonstration web service with no authentication.

The assistant definition enables Code Interpreter which the assistant uses to create a file which is downloaded in the last step.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.1"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.0

## List the assistants in your deployment
If you get an error, verify your setup!

In [7]:
var assistants = (await client.AssistantList())?.Data;
if (assistants == null)
{
    Console.WriteLine("No assistants found, check your configuration");
    return;
}

foreach(var assistant in assistants)
{
    Console.WriteLine(assistant.Name);
}

<null>

In [9]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Cat Facts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find cat facts");
}

Didn't find cat facts


## Create the Cat Facts assistant
The definition of this assistant is located in the ".\AssistantDefinitions\Cat Facts" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Cat Facts" contains the following files:
```
Cat Facts
│   instructions.md - The assistant's instructions. This file is optional but when present overwrites the value of the instructions property in manifest.json
│   manifest.json - The assistants base settings including model, "model": "gpt-4o". You need to ensure you have deployed whatever models your assistants require
│
└───OpenAPI
        cat-fact.herokuapp.com.json - The swagger definition for the test service

```

[See https://github.com/alexwohlbruck/cat-facts](https://github.com/alexwohlbruck/cat-facts)

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the cat-facts swagger which follows an Open API schema into tool defintions for the assistant.

In [10]:
var assistantId = await AssistantUtility.Create("Cat Facts", config);
var assistantId2 = await AssistantUtility.Create("ConversationUserProxy",config);
Console.WriteLine(assistantId);
Console.WriteLine(assistantId2);

asst_QaFUzkjE2WEW7wMiwvgjK7v3
asst_u2TkBjRXhlnOtnYIQKVYVuin


## Run the Assistant

`output.Dialog` shows the conversation. This version of AntRunner uses a built-in assistant definition. Upon completion the thread is deleted.

In [2]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Cat Facts",
    Instructions = "Create a plan to tell me something about cats. Review with me before using any tools",
    Evaluator = "ConversationUserProxy"
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Create a plan to tell me something about cats. Review with me before using any tools

Assistant: Sure! Here's a plan to tell you something interesting about cats:

1. **Introduction**:
   - Start with a general introduction about cats as popular pets.

2. **Cat Facts**:
   - Provide some basic facts about cats, such as their physical characteristics, behavior, and common breeds.

3. **Unique Behaviors**:
   - Discuss some unique behaviors and traits of cats, such as purring, kneading, and their grooming habits.

4. **Health and Lifespan**:
   - Share information about the average health and lifespan of domestic cats, including common health issues.

5. **Fun Facts**:
   - Provide some fun and lesser-known facts about cats to make the information engaging.

6. **Conclusion**:
   - Summarize the key points and wrap up the discussion.

Would you like to proceed with this plan, or would you like to make any changes or additions before we proceed?

User: Yes, go ahead

Assistant: Great! Let's start with the plan.

### Introduction
Cats are one of the most popular pets in the world, known for their playful and independent nature. They have been domesticated for thousands of years and are beloved by many for their companionship and unique personalities.

### Cat Facts
1. **Physical Characteristics**:
   - Cats are small, carnivorous mammals with soft fur, sharp claws, and keen senses.
   - They usually weigh between 5 to 20 pounds and have an average length of about 18 inches, excluding the tail.

2. **Behavior**:
   - Cats are known for their agility and grace. They can jump up to six times their body length.
   - They are crepuscular, meaning they are most active during dawn and dusk.

3. **Common Breeds**:
   - Some popular cat breeds include the Siamese, Persian, Maine Coon, and Bengal.

### Unique Behaviors
1. **Purring**:
   - Cats purr for various reasons, including contentment, comfort, and even when they are in pain as a self-soothing mechanism.
   
2. **Kneading**:
   - Cats often knead with their paws on soft surfaces, which is a behavior they carry from kittenhood when they kneaded their mother's belly to stimulate milk flow.

3. **Grooming**:
   - Cats spend a significant amount of time grooming themselves to keep their fur clean and free of parasites. Grooming also helps them regulate their body temperature.

### Health and Lifespan
1. **Lifespan**:
   - The average lifespan of a domestic cat is around 12 to 15 years, although some cats can live into their 20s with proper care.

2. **Common Health Issues**:
   - Common health issues in cats include dental problems, obesity, kidney disease, and hyperthyroidism. Regular veterinary check-ups are essential to maintaining a cat's health.

### Fun Facts
1. Cats have a specialized collarbone that allows them to always land on their feet when they fall.
2. A cat's nose print is unique, similar to a human fingerprint.
3. Cats have a third eyelid, called the haw, which helps protect their eyes.

### Conclusion
Cats are fascinating creatures with a rich history and a variety of unique behaviors and traits. They make wonderful companions for those who appreciate their independent yet affectionate nature.

If you'd like more detailed information on any specific aspect of cats, please let me know!


## output.LastMessage
A core perspective of AntRunnerLib is to get the richness of the assistants API with file retrieval, code interpreter, and function calls in the logical form of a chat completion.
```
output.LastMessage // Final answer
```


In [3]:
output.LastMessage.DisplayAs("text/markdown");

Great! Let's start with the plan.

### Introduction
Cats are one of the most popular pets in the world, known for their playful and independent nature. They have been domesticated for thousands of years and are beloved by many for their companionship and unique personalities.

### Cat Facts
1. **Physical Characteristics**:
   - Cats are small, carnivorous mammals with soft fur, sharp claws, and keen senses.
   - They usually weigh between 5 to 20 pounds and have an average length of about 18 inches, excluding the tail.

2. **Behavior**:
   - Cats are known for their agility and grace. They can jump up to six times their body length.
   - They are crepuscular, meaning they are most active during dawn and dusk.

3. **Common Breeds**:
   - Some popular cat breeds include the Siamese, Persian, Maine Coon, and Bengal.

### Unique Behaviors
1. **Purring**:
   - Cats purr for various reasons, including contentment, comfort, and even when they are in pain as a self-soothing mechanism.
   
2. **Kneading**:
   - Cats often knead with their paws on soft surfaces, which is a behavior they carry from kittenhood when they kneaded their mother's belly to stimulate milk flow.

3. **Grooming**:
   - Cats spend a significant amount of time grooming themselves to keep their fur clean and free of parasites. Grooming also helps them regulate their body temperature.

### Health and Lifespan
1. **Lifespan**:
   - The average lifespan of a domestic cat is around 12 to 15 years, although some cats can live into their 20s with proper care.

2. **Common Health Issues**:
   - Common health issues in cats include dental problems, obesity, kidney disease, and hyperthyroidism. Regular veterinary check-ups are essential to maintaining a cat's health.

### Fun Facts
1. Cats have a specialized collarbone that allows them to always land on their feet when they fall.
2. A cat's nose print is unique, similar to a human fingerprint.
3. Cats have a third eyelid, called the haw, which helps protect their eyes.

### Conclusion
Cats are fascinating creatures with a rich history and a variety of unique behaviors and traits. They make wonderful companions for those who appreciate their independent yet affectionate nature.

If you'd like more detailed information on any specific aspect of cats, please let me know!

## Downloading files from code interpreter
In addition to the Cat Facts API, this assistant also uses code interpreter based on its [manifest.json](.\AssistantDefinitions\Cat%20Facts\manifest.json):

```json
{
  "name": "Cat Facts",
  "description": "I look up facts about cats and can also write Python code!",
  "instructions": "",
  "tools": [
    { "type": "code_interpreter" }
  ],
  "tool_resources": {},
  "top_p": 0.95,
  "response_format": "auto",
  "metadata": {
  },
  "model": "gpt-4o",
  "temperature": 0.7
}
```

This prompt instructs the assistant to create a file **"Tell me something about cats and put the results into a file I can download"**. 

As a result the output contains Annotations. You can download the ones with annotation.Type == "file_path" as demonstrated below.

In [6]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Cat Facts",
    Instructions = "Tell me something about cats and put the results into a file I can download",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");
Console.WriteLine(output.Annotations[0].FilePathAnnotation.FileId);

// Download files specified in FilePathAnnotation
foreach (var annotation in output.Annotations)
{
    if (annotation.Type == "file_path")
    {
        var fileName = await annotation.GetFileName(config);

        var filePath = Path.Combine(@".", fileName);
        await annotation.DownloadFile(filePath, config);
        Console.WriteLine($"{filePath}");
    }
}



User: Tell me something about cats and put the results into a file I can download

Assistant: Sure! Here are some interesting facts about cats:

1. **Domestication**: Cats were first domesticated around 9,000 years ago in the Near East.
2. **Sleep**: Cats sleep for 70% of their lives, which amounts to around 13-16 hours a day.
3. **Communication**: Cats use a variety of vocalizations, body language, and scent to communicate with humans and other cats.
4. **Whiskers**: Cats have whiskers on the backs of their front legs, as well as on their faces, to help them sense their environment.
5. **Diet**: Cats are obligate carnivores, meaning they require a diet primarily of meat.
6. **Physical Abilities**: Cats have powerful night vision, allowing them to see at light levels six times lower than what a human needs.
7. **Purring**: While purring is often associated with contentment, cats also purr when they are frightened or in pain as a self-soothing mechanism.
8. **Climbing**: Cats have retractable claws that they use for climbing and catching prey.
9. **Territory**: Cats are territorial animals and often mark their territory with scent glands located on their face, paws, and tail.
10. **Lifespan**: Domestic cats can live up to 15 years or more, with some reaching their early 20s.

Now I will save these facts to a file for you to download.

I ran this code:
```
# Writing the cat facts to a file
cat_facts = """
1. Domestication: Cats were first domesticated around 9,000 years ago in the Near East.
2. Sleep: Cats sleep for 70% of their lives, which amounts to around 13-16 hours a day.
3. Communication: Cats use a variety of vocalizations, body language, and scent to communicate with humans and other cats.
4. Whiskers: Cats have whiskers on the backs of their front legs, as well as on their faces, to help them sense their environment.
5. Diet: Cats are obligate carnivores, meaning they require a diet primarily of meat.
6. Physical Abilities: Cats have powerful night vision, allowing them to see at light levels six times lower than what a human needs.
7. Purring: While purring is often associated with contentment, cats also purr when they are frightened or in pain as a self-soothing mechanism.
8. Climbing: Cats have retractable claws that they use for climbing and catching prey.
9. Territory: Cats are territorial animals and often mark their territory with scent glands located on their face, paws, and tail.
10. Lifespan: Domestic cats can live up to 15 years or more, with some reaching their early 20s.
"""

# Save to file
file_path = "/mnt/data/cat_facts.txt"
with open(file_path, "w") as file:
    file.write(cat_facts)

file_path
```

I have saved the cat facts to a file. You can download it using the link below:

[Download Cat Facts](sandbox:/mnt/data/cat_facts.txt)


assistant-dLUafkXTBnWY5HRntbBG6ccx
./cat_facts.txt


## Clean up

In [11]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Cat Facts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find Cat Facts");
}

Deleted assistant
